In [ ]:
import streamlit as st
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import pandas as pd

def evaluate_models_selected_coin_1(selected_data, chosen_model='all'):
    # Making a copy of the slice to ensure it's a separate object
    selected_data = selected_data.copy()

    for lag in range(1, 4):  # Adding lagged features for 1 to 3 days
         selected_data.loc[:, f'{selected_data.columns[0]}_lag_{lag}'] = selected_data[selected_data.columns[0]].shift(lag)

    # Dropping rows with NaN values created due to shifting
    selected_data.dropna(inplace=True)

    # Features will be the lagged values, and the target will be the current price of the first coin
    features = [f'{selected_data.columns[0]}_lag_{lag}' for lag in range(1, 4)]
    X = selected_data[features]
    y = selected_data[selected_data.columns[0]]

    # Splitting the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize dictionary to hold models
    models = {
        'GRADIENT BOOSTING': GradientBoostingRegressor(),
        'SVR': SVR(),
        'XGBOOST': XGBRegressor(),
        'LSTM': Sequential([LSTM(units=50, input_shape=(X_train.shape[1], 1)), Dense(units=1)])
    }

    eval_metrics = {}

    if chosen_model.lower() == 'all':
        chosen_models = models.keys()
    else:
        chosen_models = [chosen_model.upper()]  # Capitalize input for case insensitivity

    for model_name in chosen_models:
        if model_name not in models:
            print(f"Model '{model_name}' not found. Skipping...")
            continue

        model = models[model_name]

        if model_name == 'LSTM':
            model_filename = "Model_SELECTED_COIN_1//lstm_model.pkl"
            if os.path.exists(model_filename):
                model = load_model(model_filename)
                # Reshape the input data for LSTM model
                X_test_array = X_test.to_numpy().reshape(X_test.shape[0], X_test.shape[1], 1)
                predictions = model.predict(X_test_array).flatten()
            else:
                print("No pre-trained LSTM model found.")
                continue  # Skip the rest of the loop if LSTM model not found
        else:
            model.fit(X_train, y_train)  # Fit the model
            predictions = model.predict(X_test)

        # Calculate evaluation metrics
        mae = mean_absolute_error(y_test, predictions)
        mse = mean_squared_error(y_test, predictions)
        rmse = np.sqrt(mse)
        mape = np.mean(np.abs((y_test - predictions) / y_test)) * 100
        r2 = r2_score(y_test, predictions)

        # Store evaluation metrics
        eval_metrics[model_name] = {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'MAPE': mape, 'R2': r2}

    # Display evaluation metrics
    st.subheader("Evaluation Metrics")
    for model_name, metrics in eval_metrics.items():
        st.write(f"Evaluation metrics for {model_name}:")
        for metric_name, value in metrics.items():
            st.write(f"{metric_name}: {value}")
        st.write()

    # Plot evaluation metrics
    if eval_metrics:
        st.subheader("Evaluation Metric Visualization")
        fig, ax = plt.subplots(figsize=(12, 6))

        metrics = list(eval_metrics.keys())
        mae_values = [eval_metrics[model]['MAE'] for model in metrics]  # Corrected access to values
        mse_values = [eval_metrics[model]['MSE'] for model in metrics]  # Corrected access to values
        rmse_values = [eval_metrics[model]['RMSE'] for model in metrics]  # Corrected access to values

        bar_width = 0.15  # Increased bar width
        index = np.arange(len(metrics))

        bar1 = ax.bar(index - 2*bar_width, mae_values, bar_width, label='MAE')  # Adjusted positions for bars
        bar2 = ax.bar(index - bar_width, mse_values, bar_width, label='MSE')   # Adjusted positions for bars
        bar3 = ax.bar(index, rmse_values, bar_width, label='RMSE')              # Adjusted positions for bars

        ax.set_xlabel('Models')
        ax.set_ylabel('Metrics')
        ax.set_title(f'Evaluation Metrics for {selected_data.columns[0]} using {chosen_model.upper()} as the Models')
        ax.set_xticks(index)
        ax.set_xticklabels(metrics)
        ax.legend()

        # Annotate bars with values
        for bars in [bar1, bar2, bar3]:
            for bar in bars:
                height = bar.get_height()
                ax.annotate('{}'.format(round(height, 2)),
                            xy=(bar.get_x() + bar.get_width() / 2, height),
                            xytext=(0, 3),  # 3 points vertical offset
                            textcoords="offset points",
                            ha='center', va='bottom')

        st.pyplot(fig)
    else:
        st.write("No models were evaluated.")

# Load the selected data from a CSV file
selected_data = pd.read_csv("Selected_coins.csv", index_col='Date')

# Streamlit app
st.title("Model Evaluation for Selected Coin 1")
chosen_model = st.selectbox("Choose the model you want to evaluate (or select 'all' for all models)", ['all', 'Gradient Boosting', 'SVR', 'XGBoost', 'LSTM'])

if chosen_model != 'all':
    evaluate_models_selected_coin_1(selected_data, chosen_model)
else:
    for model in ['Gradient Boosting', 'SVR', 'XGBoost', 'LSTM']:
        evaluate_models_selected_coin_1(selected_data, model)


In [ ]:
# Evaluate the model and get the trained model and selected_data
#     model, selected_data = evaluate_models_BTC(selected_data)

#     if model is not None:
#         # User input for frequency and number of periods (weeks, months, or quarters)
#         frequency = st.sidebar.selectbox("Frequency", ['daily', 'weekly', 'monthly', 'quarterly'])
#         num_periods = st.sidebar.number_input("Number of Periods", min_value=1, step=1)

#         # Make predictions for the specified number of periods
#         features = [f'BTC-GBP_lag_{lag}' for lag in range(1, 4)]
#         X_array = selected_data[features].to_numpy()
#         predictions = model.predict(X_array[-num_periods:])  # Predictions for the last 'num_periods' rows

#         # Get the last date in the dataset
#         last_date = selected_data.index[-1]

#         # Generate periods for future predictions
#         if frequency == 'daily':
#             periods = pd.date_range(start=last_date, periods=num_periods, freq='D')
#         elif frequency == 'weekly':
#             periods = pd.date_range(start=last_date, periods=num_periods, freq='W')
#         elif frequency == 'monthly':
#             periods = pd.date_range(start=last_date, periods=num_periods, freq='M')
#         elif frequency == 'quarterly':
#             periods = pd.date_range(start=last_date, periods=num_periods, freq='Q')
#         else:
#             st.write("Invalid frequency. Please choose from 'daily', 'weekly', 'monthly', or 'quarterly'.")

#         # Plot average prices with confidence intervals
#         mse = mean_squared_error(y_test[-num_periods:], predictions)
#         plot_average_prices(predictions, periods, frequency, mse)

#         # Plot actual and forecasted prices with confidence intervals
#         upper_bound = predictions + 1.96 * np.sqrt(mse)
#         lower_bound = predictions - 1.96 * np.sqrt(mse)
#         plot_actual_forecast_with_confidence(y_test[-num_periods:], predictions, periods, upper_bound, lower_bound)

# #     elif visualization_option == "BoxPlot":
# #         # Display coin selection dropdown
# #         selected_coin = st.sidebar.selectbox("Select a cryptocurrency:", data['Crypto'].unique())

#         # Plot boxplot for the selected coin
#         # plot_boxplot(data, selected_coin)

In [ ]:
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

# Define function to evaluate models for BTC
def evaluate_models_BTC(selected_data):
    # Making a copy of the slice to ensure it's a separate object
    selected_data = pd.DataFrame(selected_data)

    for lag in range(1, 4):  # Adding lagged features for 1 to 3 days
        selected_data.loc[:, f'BTC-GBP_lag_{lag}'] = selected_data['BTC-GBP'].shift(lag)

    # Dropping rows with NaN values created due to shifting
    selected_data.dropna(inplace=True)

    # Features will be the lagged values, and the target will be the current BTC-GBP price
    features = [f'BTC-GBP_lag_{lag}' for lag in range(1, 4)]
    X = selected_data[features]
    y = selected_data['BTC-GBP']

    # Splitting the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # User input for selecting the model
    model_choice = st.sidebar.selectbox("Choose Model", ['GBR', 'SVR', 'XGB', 'LSTM'])

    # Initialize and train the selected model
    if model_choice == 'GBR':
        model = GradientBoostingRegressor()
        params = {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.01, 0.1, 0.5],
            'max_depth': [3, 5, 7],
            'min_samples_leaf': [1, 2, 4],
            'subsample': [0.8, 0.9, 1.0]
        }
    elif model_choice == 'SVR':
        model = SVR()
        params = {
            'C': [0.1, 1, 10],
            'kernel': ['linear', 'rbf', 'poly'],
            'gamma': ['scale', 'auto']
        }
    elif model_choice == 'XGB':
        model = XGBRegressor()
        params = {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.01, 0.1, 0.5],
            'max_depth': [3, 5, 7],
            'subsample': [0.8, 0.9, 1.0]
        }
    elif model_choice == 'LSTM':
        model = Sequential()
        model.add(LSTM(units=50, input_shape=(X_train.shape[1], 1)))
        model.add(Dense(units=1))
        model.compile(optimizer='adam', loss='mean_squared_error')

        X_train_array = X_train.to_numpy()
        X_test_array = X_test.to_numpy()

        X_train_lstm = X_train_array.reshape(X_train_array.shape[0], X_train_array.shape[1], 1)
        X_test_lstm = X_test_array.reshape(X_test_array.shape[0], X_test_array.shape[1], 1)

        model.fit(X_train_lstm, y_train, epochs=100, batch_size=32)
        return None, None, None  # Return None as predictions, periods, and y_test for LSTM model

    else:
        st.write("Invalid model choice. Please choose from GBR, SVR, XGB, or LSTM.")
        return None, None, None

    grid_search = GridSearchCV(estimator=model, param_grid=params, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)

    best_params = grid_search.best_params_
    st.write(f"Best parameters for {model_choice}:", best_params)

    model_best = model.set_params(**best_params)

    # Impute missing values using mean imputation
    imputer = SimpleImputer(strategy='mean')
    X_train_imputed = imputer.fit_transform(X_train)
    X_test_imputed = imputer.transform(X_test)

    # Fit the model using imputed data
    model_best.fit(X_train_imputed, y_train)

    return model_best, selected_data, y_test

# Define function to plot average prices
def plot_average_prices(predictions, periods, frequency, mse):
    # Convert predictions and periods to DataFrame
    df = pd.DataFrame({'Period': periods, 'Prediction': predictions})
    df.set_index('Period', inplace=True)

    # Resample to the desired frequency and calculate the mean
    if frequency == 'daily':
        df_resampled = df
    elif frequency == 'weekly':
        df_resampled = df.resample('W').mean()
    elif frequency == 'monthly':
        df_resampled = df.resample('M').mean()
    elif frequency == 'quarterly':
        df_resampled = df.resample('Q').mean()
    else:
        st.write("Invalid frequency. Please choose from 'daily', 'weekly', 'monthly', or 'quarterly'.")
        return

    # Plot the average prices
    plt.plot(df_resampled.index, df_resampled['Prediction'], marker='o')
    plt.xlabel('Date')
    plt.ylabel('Average BTC Price')
    plt.title(f'Average Predicted BTC Price ({frequency.capitalize()})')
    plt.grid(True)
    
    # Set x-axis tick labels
    plt.xticks(df_resampled.index, rotation=45, labels=df_resampled.index.strftime('%Y-%m-%d'))

    st.pyplot()

# Define function to plot actual and forecasted prices with confidence intervals
def plot_actual_forecast_with_confidence(actual, forecast, periods, upper_bound, lower_bound):
    plt.figure(figsize=(10, 6))
    plt.plot(periods, actual, label='Actual', color='blue')
    plt.plot(periods, forecast, label='Forecast', color='green')
    plt.fill_between(periods, upper_bound, lower_bound, color='lightgray', alpha=0.5, label='Confidence Interval')
    plt.xlabel('Date')
    plt.ylabel('BTC Price')
    plt.title('Actual vs Forecasted BTC Price with Confidence Interval')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)

    st.pyplot()

# Sidebar navigation
side_bars = st.sidebar.radio("Navigation", ["Home", "About Us", "Dataset", "Coin Correlation", "Moving Average", "Visualizations", "Predictions"])

# Conditionals for sidebar navigation
if side_bars == "Home":
    st.write("Welcome to the Home page!")
elif side_bars == "About Us":
    st.write("This is the About Us page!")
elif side_bars == "Dataset":
    st.write("You are on the Dataset page!")
elif side_bars == "Coin Correlation":
    st.write("You are on the Coin Correlation page!")
elif side_bars == "Moving Average":
    st.write("You are on the Moving Average page!")
elif side_bars == "Visualizations":
    st.write("You are on the Visualizations page!")
elif side_bars == "Predictions":
    st.write("You are on the Predictions page!")
    # Evaluate the model and get the trained model and selected_data
    model, selected_data, y_test = evaluate_models_BTC(selected_data)

    if model is not None:
        # User input for frequency and number of periods (weeks, months, or quarters)
        frequency = st.sidebar.selectbox("Frequency", ['daily', 'weekly', 'monthly', 'quarterly'])
        num_periods = st.sidebar.number_input("Number of Periods", min_value=1, step=1)

        # Make predictions for the specified number of periods
        features = [f'BTC-GBP_lag_{lag}' for lag in range(1, 4)]
        X_array = selected_data[features].to_numpy()
        predictions = model.predict(X_array[-num_periods:])  # Predictions for the last 'num_periods' rows

        # Get the last date in the dataset
        last_date = selected_data.index[-1]

        # Generate periods for future predictions
        if frequency == 'daily':
            periods = pd.date_range(start=last_date, periods=num_periods, freq='D')
        elif frequency == 'weekly':
            periods = pd.date_range(start=last_date, periods=num_periods, freq='W')
        elif frequency == 'monthly':
            periods = pd.date_range(start=last_date, periods=num_periods, freq='M')
        elif frequency == 'quarterly':
            periods = pd.date_range(start=last_date, periods=num_periods, freq='Q')
        else:
            st.write("Invalid frequency. Please choose from 'daily', 'weekly', 'monthly', or 'quarterly'.")

        # Plot average prices with confidence intervals
        mse = mean_squared_error(y_test[-num_periods:], predictions)
        plot_average_prices(predictions, periods, frequency, mse)

        # Plot actual and forecasted prices with confidence intervals
        upper_bound = predictions + 1.96 * np.sqrt(mse)
        lower_bound = predictions - 1.96 * np.sqrt(mse)
        plot_actual_forecast_with_confidence(y_test[-num_periods:], predictions, periods, upper_bound, lower_bound)


In [ ]:
import streamlit as st
import pandas as pd
import numpy as np

# Assuming 'data' is loaded elsewhere in your Streamlit app, for example:
# data = pd.read_csv('combined_crypto_data.csv')

def analyze_coin_correlation(data):
    
    
    st.header("Coin Correlation Analysis")
    
    st.write("Streamlit interface for analyzing and displaying the top four positively and negatively correlated cryptocurrencies with user-selected coins.")

    coins_list = data.columns.tolist()
    st.write("Available coins:", ', '.join(coins_list))

    # Streamlit user input for selecting multiple coins
    coins_selected = st.multiselect("Select the coins you want to analyze:", coins_list)

    for coin_selected in coins_selected:
        st.subheader(f"Analysis for {coin_selected}")
        # Prices for the selected coin
        selected_coin_prices = data[coin_selected]

        # Calculate correlations with the selected coin
        correlations = data.corrwith(selected_coin_prices)

        # Sort correlations in descending order, excluding the selected coin
        sorted_correlations = correlations.sort_values(ascending=False).drop(coin_selected)

        # Get top four positively and negatively correlated cryptocurrencies
        positively_correlated = sorted_correlations.head(4)
        negatively_correlated = sorted_correlations.tail(4)

        st.write(f"Top four positively correlated cryptocurrencies with {coin_selected}:")
        st.dataframe(positively_correlated)

        st.write(f"\nTop four negatively correlated cryptocurrencies with {coin_selected}:")
        st.dataframe(negatively_correlated)

# Example usage within Streamlit
if side_bars == "Coin Correlation":
    # Assuming 'data' is prepared as required
    analyze_coin_correlation(data)


In [ ]:
import streamlit as st

def home_section():
    st.header("SOLiGence")
    st.write(
        "Solent Intelligence (SOLiGence) is a leading financial multinational organisation that deals"
        " with stock and shares, saving and investments.")

    # Streamlit app
    st.title('Welcome to SOLiGence')
    st.write("Your Intelligent Coin Trading Platform")

    # Brief Description
    st.write("Empower your cryptocurrency trading decisions with AI-driven insights and real-time data.")

    # Call to Action (CTA) Button
    if st.button("Get Started"):
        st.write("Let's explore the world of cryptocurrency trading together!")

        # Background Image
        st.image('sal.jpg', use_column_width=True)
        if 'section' not in st.session_state:
            st.session_state.section = "Home"

        # News and Updates
        st.write("News and Updates:")
        st.info("Stay tuned for the latest updates and trends in the cryptocurrency market!")

        # Testimonials
        st.write("What Our Users Say:")
        st.write("The SOLiGence app transformed how I approach cryptocurrency trading. Highly recommended!")

        # Contact Information
        st.write("Contact Us:")
        st.write("For inquiries, email us at info@soligence.com")

        # Social Media Links
        st.write("Connect with Us:")
        st.markdown("[Twitter](https://twitter.com/SOLiGenceApp) [LinkedIn](https://linkedin.com/company/soligence)")

        # Privacy and Disclaimer
        st.write("Privacy Policy | Disclaimer")

# Streamlit app
st.title('Welcome to SOLiGence')

# Sidebar navigation
side_bars = st.sidebar.radio("Navigation", ["Home", "About Us", "Dataset","Coin Correlation","Moving Average", "Visualizations", "Predictions"])

# Conditionals for sidebar navigation
if side_bars == "Home":
    home_section()


In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np

# Load cryptocurrency price data
data = pd.read_csv('combined_crypto_data.csv')

def analyze_coin_correlation(data):
    """
    Streamlit interface for analyzing and displaying the top four positively and 
    negatively correlated cryptocurrencies with user-selected coins.
    """
    st.header("Coin Correlation Analysis")
    st.write("Streamlit interface for analyzing and displaying the top four positively and negatively correlated cryptocurrencies with user-selected coins.")

    coins_list = data.columns.tolist()
    st.write("Available coins:", ', '.join(coins_list))

    coins_selected = st.multiselect("Select the coins you want to analyze:", coins_list)

    for coin_selected in coins_selected:
        st.subheader(f"Analysis for {coin_selected}")
        selected_coin_prices = data[coin_selected]
        correlations = data.corrwith(selected_coin_prices)
        sorted_correlations = correlations.sort_values(ascending=False).drop(coin_selected)
        positively_correlated = sorted_correlations.head(4)
        negatively_correlated = sorted_correlations.tail(4)

        st.write(f"Top four positively correlated cryptocurrencies with {coin_selected}:")
        st.dataframe(positively_correlated)
        st.write(f"Top four negatively correlated cryptocurrencies with {coin_selected}:")
        st.dataframe(negatively_correlated)

def about_us():
    st.title("About Solent Intelligence Ltd.")
    st.write(
        "The scale of this organization's operation is impressive, with millions of subscribers and over 150 billion "
        "pounds worth of investments. This emphasizes the substantial influence that data-driven decisions can have "
        "on managing such a significant amount of assets. The app's focus on implementing an Intelligent Coin Trading "
        "(IST) platform, specifically tailored for crypto coin predictions, resonates deeply with me. The idea of "
        "utilizing AI to predict cryptocurrency prices on different timeframes, such as daily, weekly, monthly, "
        "and quarterly, truly piques my interest. This approach aligns perfectly with my desire to explore how AI can "
        "shape and enhance our daily lives. The app's ability to recommend trading opportunities by analyzing "
        "AI-generated predictions showcases the tangible applications of data science in the financial world. "
        "Considering a more neutral perspective, while the concept of the app is exciting, there are potential "
        "challenges that need to be acknowledged. Cryptocurrency markets are known for their volatility, and even the "
        "most sophisticated AI predictions might not always be entirely accurate. Users relying solely on these "
        "predictions could face risks if market conditions change unexpectedly.")

def dataset():
    st.header("Crypto Dataset of Five Coins")

    # Sidebar options
    st.sidebar.subheader("Dataset Options")

    # Sorting
    sort_column = st.sidebar.multiselect("Sort by:", data.columns)
    ascending = st.sidebar.checkbox("Ascending")
    sorted_data = data.sort_values(by=sort_column, ascending=ascending)

    # Filtering
    selected_crypto = st.sidebar.selectbox("Filter by cryptocurrency:", ["All"] + data['Crypto'].unique())
    if "All" not in selected_crypto:
        sorted_data = sorted_data[sorted_data['Crypto'].isin(selected_crypto)]

    # Pagination
    page_size = st.sidebar.number_input("Items per page:", min_value=1, value=10)
    page_number = st.sidebar.number_input("Page number:", min_value=1, value=1)
    start_idx = (page_number - 1) * page_size
    end_idx = start_idx + page_size
    paginated_data = sorted_data.iloc[start_idx:end_idx]

    # Display the dataset using a dataframe
    st.subheader("Dataset Overview")
    st.dataframe(paginated_data)

    # Provide an option to show a table view
    show_table = st.checkbox("Show as Table")

    # Display the dataset as a table if the checkbox is selected
    if show_table:
        st.subheader("Dataset Table View")
        st.table(paginated_data)

# Streamlit app
st.title('Welcome to SOLiGence')
st.write("Your Intelligent Coin Trading Platform")

# Logo
# st.image('your_logo.png', width=200)

# Brief Description
st.write("Empower your cryptocurrency trading decisions with AI-driven insights and real-time data.")

# Call to Action (CTA) Button
if st.button("Get Started"):
    st.write("Let's explore the world of cryptocurrency trading together!")

    # Background Image
    st.image('sal.jpg', use_column_width=True)
    if 'section' not in st.session_state:
        st.session_state.section = "Home"

    # News and Updates
    st.write("News and Updates:")
    st.info("Stay tuned for the latest updates and trends in the cryptocurrency market!")

    # Testimonials
    st.write("What Our Users Say:")
    st.write("The SOLiGence app transformed how I approach cryptocurrency trading. Highly recommended!")

    # Contact Information
    st.write("Contact Us:")
    st.write("For inquiries, email us at info@soligence.com")

    # Social Media Links
    st.write("Connect with Us:")
    st.markdown("[Twitter](https://twitter.com/SOLiGenceApp) [LinkedIn](https://linkedin.com/company/soligence)")

    # Privacy and Disclaimer
    st.write("Privacy Policy | Disclaimer")

# Sidebar navigation
side_bars = st.sidebar.radio("Navigation", ["Home", "About Us", "Dataset","Coin Correlation","Moving Average", "Visualizations", "Predictions"])

# Conditionals for sidebar navigation
if side_bars == "About Us":
    about_us()
elif side_bars == "Dataset":
    dataset()
elif side_bars == "Coin Correlation":
    analyze_coin_correlation(data)


In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt

def plot_crypto_prices(crypto_data, coins, metrics, start_date, end_date):
    """
    Plots the price metrics of selected cryptocurrencies over a specified interval.

    Parameters:
    - crypto_data (pd.DataFrame): DataFrame containing the cryptocurrency data with date as index.
    - coins (list): List of cryptocurrencies to plot.
    - metrics (list): List of price metrics to plot (e.g., 'Close', 'Open', 'High', 'Low').
    - start_date (str): Start date of the interval in 'YYYY-MM-DD' format.
    - end_date (str): End date of the interval in 'YYYY-MM-DD' format.
    """
    # Filter data for the selected coins and interval
    selected_data = crypto_data[(crypto_data['Crypto'].isin(coins)) & (crypto_data.index >= start_date) & (crypto_data.index <= end_date)]

    # Plot the selected price metrics for each coin
    plt.figure(figsize=(12, 6))
    for coin in coins:
        for metric in metrics:
            plt.plot(selected_data.index, selected_data.loc[selected_data['Crypto'] == coin, metric], label=f"{coin} {metric}")

    # Add labels and title
    plt.title('Cryptocurrency Prices Over Time')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)
    
    # Show plot
    plt.tight_layout()
    st.pyplot(plt)

# Example usage
# Assuming 'crypto_data' is your DataFrame containing cryptocurrency data
# and it has columns: 'Crypto', 'Date', 'Open', 'High', 'Low', 'Close'

if side_bars == "Visualizations":
    st.title('Cryptocurrency Price Visualization')
    st.subheader('Plot Crypto Prices Over Time')

    # User input
    coins = st.multiselect("Select the cryptocurrencies you want to plot:", crypto_data['Crypto'].unique())
    metrics = st.multiselect("Select the price metrics you want to plot:", ['Close', 'Open', 'High', 'Low'], default=['Close'])
    start_date = st.date_input("Enter the start date of the interval:", crypto_data.index.min())
    end_date = st.date_input("Enter the end date of the interval:", crypto_data.index.max())

    # Plot the selected cryptocurrency prices
    plot_crypto_prices(crypto_data, coins, metrics, str(start_date), str(end_date))


In [2]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_crypto_prices(crypto_data, coins, metrics, start_date, end_date):
    """
    Plots the price metrics of selected cryptocurrencies over a specified interval.

    Parameters:
    - crypto_data (pd.DataFrame): DataFrame containing the cryptocurrency data with date as index.
    - coins (list): List of cryptocurrencies to plot.
    - metrics (list): List of price metrics to plot (e.g., 'Close', 'Open', 'High', 'Low').
    - start_date (str): Start date of the interval in 'YYYY-MM-DD' format.
    - end_date (str): End date of the interval in 'YYYY-MM-DD' format.
    """
    # Convert start_date and end_date to datetime objects
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Filter data for the selected coins and interval
    selected_data = crypto_data[(crypto_data['Crypto'].isin(coins)) & 
                                (crypto_data.index >= start_date) & 
                                (crypto_data.index <= end_date)]

    # Plot the selected price metrics for each coin
    plt.figure(figsize=(12, 6))
    for coin in coins:
        for metric in metrics:
            plt.plot(selected_data.index, 
                     selected_data.loc[selected_data['Crypto'] == coin, metric], 
                     label=f"{coin} {metric}")

    # Add labels and title
    plt.title('Cryptocurrency Prices Over Time')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)
    
    # Show plot
    plt.tight_layout()
    plt.show()

# # Example data
crypto_data = crypto_data.copy()

# pd.DataFrame({
#     'Crypto': ['BTC', 'BTC', 'ETH', 'ETH'],
#     'Date': pd.date_range('2022-01-01', periods=4, freq='D'),
#     'Open': [30000, 31000, 1500, 1600],
#     'High': [31000, 32000, 1600, 1700],
#     'Low': [29000, 30000, 1400, 1500],
#     'Close': [30500, 31500, 1550, 1650]
# })
# crypto_data.set_index('Date', inplace=True)

# User input
coins = input("Enter the cryptocurrencies you want to plot (comma-separated): ").split(',')
metrics = input("Enter the price metrics you want to plot (comma-separated, e.g., Close,Open,High,Low): ").split(',')
start_date = input("Enter the start date of the interval (YYYY-MM-DD): ")
end_date = input("Enter the end date of the interval (YYYY-MM-DD): ")

# Plot the selected cryptocurrency prices
plot_crypto_prices(crypto_data, coins, metrics, start_date, end_date)


NameError: name 'crypto_data' is not defined

In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt

def plot_crypto_prices(crypto_data, compare_option, coins, metrics, start_date, end_date):
    """
    Plots the price metrics of selected cryptocurrencies over a specified interval, with optional comparisons.

    Parameters:
    - compare_option (str): Option for comparison. Can be 'coins' to compare metrics between coins, 'metrics' to compare within a coin, or 'none' for no comparison.
    - coins (list): List of cryptocurrencies to plot.
    - metrics (list): List of price metrics to plot (e.g., 'Close', 'Open', 'High', 'Low', 'Volume').
    - start_date (str): Start date of the interval in 'YYYY-MM-DD' format.
    - end_date (str): End date of the interval in 'YYYY-MM-DD' format.
    """
    # Filter data for the selected coins and interval
    selected_data = crypto_data[(crypto_data['Crypto'].isin(coins)) & (crypto_data.index >= start_date) & (crypto_data.index <= end_date)]

    # Plot the selected price metrics
    plt.figure(figsize=(12, 6))

    if compare_option == 'coins':
        for metric in metrics:
            for coin in coins:
                coin_data = selected_data[selected_data['Crypto'] == coin]
                plt.plot(coin_data.index, coin_data[metric], label=f"{coin} {metric}")
    elif compare_option == 'metrics':
        for coin in coins:
            for metric in metrics:
                coin_data = selected_data[selected_data['Crypto'] == coin]
                plt.plot(coin_data.index, coin_data[metric], label=f"{coin} {metric}")
    else:
        for coin in coins:
            for metric in metrics:
                coin_data = selected_data[selected_data['Crypto'] == coin]
                plt.plot(coin_data.index, coin_data[metric], label=f"{coin} {metric}")

    # Add labels and title
    plt.title('Cryptocurrency Prices Over Time')
    plt.xlabel('Date')
    plt.ylabel('Price/Volume')
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)
    
    # Show plot
    st.pyplot(plt)


# Streamlit UI
st.title('Cryptocurrency Price Visualization')

# User input
compare_option = st.radio("Select comparison option:", ['coins', 'metrics', 'none'])
# Load crypto data (replace this with your own data loading method)
crypto_data = pd.read_csv('crypto_data.csv')
coins = st.multiselect("Select cryptocurrencies:", crypto_data['Crypto'].unique(), default=['BTC-GBP'])
metrics = st.multiselect("Select price metrics:", ['Close', 'Open', 'High', 'Low', 'Volume'], default=['Close'])
start_date = st.date_input("Select start date:", pd.to_datetime('2023-04-08'))
end_date = st.date_input("Select end date:", pd.to_datetime('2024-04-08'))

# Plot the selected cryptocurrency prices
if st.button("Plot"):
    plot_crypto_prices(crypto_data, compare_option, coins, metrics, start_date, end_date)


In [ ]:
import streamlit as st
import pandas as pd
import numpy as np

def predict_market_state(crypto_data, selected_date, coins_selected):
    """
    Predicts the market state (up or down) for a group of chosen cryptocurrencies.

    Parameters:
    - crypto_data (pd.DataFrame): DataFrame containing the cryptocurrency data with date as index.
    - selected_date (str): The date for analysis in 'YYYY-MM-DD' format.
    - coins_selected (list): List of cryptocurrencies selected by the user.

    Returns:
    - market_state (pd.Series): Series indicating the market state for each selected cryptocurrency.
    - decision (str): Decision based on the market state prediction.
    """

    # Filter data for the selected date and coins
    selected_data = crypto_data.loc[selected_date]
    selected_data = selected_data[selected_data['Crypto'].isin(coins_selected)].copy()

    # Convert 'Close' column to numeric, dropping rows with non-numeric values
    selected_data['Close'] = pd.to_numeric(selected_data['Close'], errors='coerce')
    selected_data.dropna(subset=['Close'], inplace=True)

    # Calculate daily price changes for each coin
    price_changes = selected_data.groupby('Crypto')['Close'].pct_change()

    # Predict market state (up or down) based on overall price movement of selected coins
    market_state = np.sign(price_changes).astype(float)

    # Decision based on market state prediction
    if market_state.mean() > 0:
        decision = "The market is predicted to increase or go up for the selected cryptocurrencies on the chosen date."
    elif market_state.mean() < 0:
        decision = "The market is predicted to decrease or go down for the selected cryptocurrencies on the chosen date."
    else:
        decision = "The market is predicted to remain stable for the selected cryptocurrencies on the chosen date."

    return market_state, decision

# Load cryptocurrency data
crypto_data = pd.read_csv('crypto_data.csv')  # Replace with your data loading method

# Sidebar for user input
st.sidebar.title('Select Analysis Parameters')
selected_date = st.sidebar.date_input("Select a date for analysis:", pd.to_datetime('2024-04-08'))
coins_available = crypto_data['Crypto'].unique()
coins_selected = st.sidebar.multiselect("Select cryptocurrencies:", coins_available)

# Predict market state and decision
market_state, decision = predict_market_state(crypto_data, selected_date, coins_selected)

# Display the result and decision to the user
st.title('Cryptocurrency Market State Prediction')
st.write(f"Market state prediction for {selected_date}:")
st.write(market_state)
st.write("Based on the prediction:")
st.write(decision)


In [ ]:
import streamlit as st
import pandas as pd
import numpy as np

def predict_market_state_with_model(crypto_data, selected_date, coins_selected, model):
    """
    Predicts the market state (up or down) for a group of chosen cryptocurrencies using a trained model.

    Parameters:
    - crypto_data (pd.DataFrame): DataFrame containing the cryptocurrency data with date as index.
    - selected_date (str): The date for analysis in 'YYYY-MM-DD' format.
    - coins_selected (list): List of cryptocurrencies selected by the user.
    - model: Trained machine learning model for predicting market state.

    Returns:
    - market_state (pd.Series): Series indicating the market state for each selected cryptocurrency.
    - decision (str): Decision based on the market state prediction.
    """

    # Filter data for the selected date and coins
    selected_data = crypto_data.loc[selected_date]
    selected_data = selected_data[selected_data['Crypto'].isin(coins_selected)].copy()

    # Convert 'Close' column to numeric, dropping rows with non-numeric values
    selected_data['Close'] = pd.to_numeric(selected_data['Close'], errors='coerce')
    selected_data.dropna(subset=['Close'], inplace=True)

    # Use the trained model to predict market state
    # You would replace this with the appropriate code to use your trained model
    # For example:
    # market_state = model.predict(selected_data)

    # For demonstration purposes, let's assume random predictions
    market_state = pd.Series(np.random.choice([-1, 1], size=len(coins_selected)), index=coins_selected)

    # Decision based on market state prediction
    if market_state.mean() > 0:
        decision = "The market is predicted to increase or go up for the selected cryptocurrencies on the chosen date."
    elif market_state.mean() < 0:
        decision = "The market is predicted to decrease or go down for the selected cryptocurrencies on the chosen date."
    else:
        decision = "The market is predicted to remain stable for the selected cryptocurrencies on the chosen date."

    return market_state, decision

# Load cryptocurrency data
crypto_data = pd.read_csv('crypto_data.csv')  # Replace with your data loading method

# Sidebar for user input
st.sidebar.title('Select Analysis Parameters')
selected_date = st.sidebar.date_input("Select a date for analysis:", pd.to_datetime('2024-04-08'))
coins_available = crypto_data['Crypto'].unique()
coins_selected = st.sidebar.multiselect("Select cryptocurrencies:", coins_available)

# Load the trained model
# Replace this with your code to load the trained model
# For example:
# model = load_model('trained_model.h5')

# Predict market state and decision
market_state, decision = predict_market_state_with_model(crypto_data, selected_date, coins_selected, model)

# Display the result and decision to the user
st.title('Cryptocurrency Market State Prediction')
st.write(f"Market state prediction for {selected_date}:")
st.write(market_state)
st.write("Based on the prediction:")
st.write(decision)


In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf

# Define the predict_market_state function
def predict_market_state(crypto_data):
    """
    Predicts the market state (up or down) for a group of chosen cryptocurrencies.

    Parameters:
    - crypto_data (pd.DataFrame): DataFrame containing the cryptocurrency data with date as index.

    Returns:
    - market_state (pd.Series): Series indicating the market state for each day.
    """

    # User input for selecting cryptocurrencies
    st.sidebar.title("Market State Prediction")
    st.sidebar.write("Predicts the market state (up or down) for a group of chosen cryptocurrencies.")

    available_coins = crypto_data['Crypto'].unique()
    coins_selected = st.sidebar.multiselect("Select cryptocurrencies", available_coins)

    if coins_selected:
        selected_data = crypto_data[crypto_data['Crypto'].isin(coins_selected)].copy()

        # Calculate daily price changes for each coin
        price_changes = selected_data['Close'].pct_change()

        # Predict market state (up or down) based on overall price movement of selected coins
        market_state = pd.Series(np.sign(price_changes).astype(float), index=selected_data.index)

        # Visualize market state
        st.write("Market State:")
        st.line_chart(market_state)

        return market_state
    else:
        st.warning("Please select at least one cryptocurrency.")

# Define the other visualization functions

def plot_crypto_prices_visualization(crypto_data, compare_option, coins, metrics, start_date, end_date):
    """
    Plots the price metrics of selected cryptocurrencies over a specified interval, with optional comparisons.

    Parameters:
    - compare_option (str): Option for comparison. Can be 'coins' to compare metrics between coins, 'metrics' to compare within a coin, or 'none' for no comparison.
    - coins (list): List of cryptocurrencies to plot.
    - metrics (list): List of price metrics to plot (e.g., 'Close', 'Open', 'High', 'Low', 'Volume').
    - start_date (str): Start date of the interval in 'YYYY-MM-DD' format.
    - end_date (str): End date of the interval in 'YYYY-MM-DD' format.
    """
    # Function implementation remains unchanged

def plot_candlestick_chart(coin='BTC-GBP', period='D'):
    """
    Plots the candlestick chart for a specific cryptocurrency.

    Parameters:
    - coin (str): The cryptocurrency symbol.
    - period (str): The time period for the candlestick chart ('D' for daily, 'W' for weekly, 'M' for monthly).
    """
    # Function implementation remains unchanged

# Streamlit app starts here

# Load cryptocurrency data
crypto_data = pd.read_csv('crypto_data.csv')

# Sidebar
st.sidebar.title("Cryptocurrency Analysis")
option = st.sidebar.selectbox("Select Visualization", ["Market State Prediction", "Price Metrics Visualization", "Candlestick Chart"])

# Main content based on selected option
if option == "Market State Prediction":
    predict_market_state(crypto_data)
elif option == "Price Metrics Visualization":
    # Implement code for price metrics visualization
    pass
elif option == "Candlestick Chart":
    # Implement code for candlestick chart visualization
    pass


In [ ]:
#Streamlit visualization


import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf

def predict_market_state(crypto_data):
    """
    Predicts the market state (up or down) for a group of chosen cryptocurrencies.

    Parameters:
    - crypto_data (pd.DataFrame): DataFrame containing the cryptocurrency data with date as index.

    Returns:
    - market_state (pd.Series): Series indicating the market state for each day.
    """

    # User input for selecting cryptocurrencies
    available_coins = crypto_data['Crypto'].unique()
    coins_selected = st.sidebar.multiselect("Select cryptocurrencies to analyze", available_coins)

    # Filter data for the selected coins
    selected_data = crypto_data[crypto_data['Crypto'].isin(coins_selected)].copy()

    # Visualize market state
    st.subheader('Market State Visualization')
    fig, ax = plt.subplots(figsize=(10, 6))
    selected_data['Close'].plot(ax=ax, label='Close Price')
    ax.scatter(selected_data[selected_data['Market State'] == -1].index, 
               selected_data[selected_data['Market State'] == -1]['Close'], 
               c='r', label='Down Market', marker='v', alpha=0.5)
    ax.scatter(selected_data[selected_data['Market State'] == 1].index, 
               selected_data[selected_data['Market State'] == 1]['Close'], 
               c='g', label='Up Market', marker='^', alpha=0.5)
    ax.set_title('Market State Prediction')
    ax.set_xlabel('Date')
    ax.set_ylabel('Price')
    ax.legend()
    st.pyplot(fig)

    return market_state

def plot_crypto_prices_visualization(crypto_data, compare_option, coins, metrics, start_date, end_date):
    """
    Plots the price metrics of selected cryptocurrencies over a specified interval, with optional comparisons.

    Parameters:
    - compare_option (str): Option for comparison. Can be 'coins' to compare metrics between coins, 'metrics' to compare within a coin, or 'none' for no comparison.
    - coins (list): List of cryptocurrencies to plot.
    - metrics (list): List of price metrics to plot (e.g., 'Close', 'Open', 'High', 'Low', 'Volume').
    - start_date (str): Start date of the interval in 'YYYY-MM-DD' format.
    - end_date (str): End date of the interval in 'YYYY-MM-DD' format.
    """
    st.title('Cryptocurrency Price Metrics Visualization')
    st.write("Plots the price metrics of selected cryptocurrencies over a specified interval, with optional comparisons")

    # Filter data for the selected coins and interval
    selected_data = crypto_data[(crypto_data['Crypto'].isin(coins)) & (crypto_data.index >= start_date) & (crypto_data.index <= end_date)]

    # Plot the selected price metrics
    plt.figure(figsize=(12, 6))

    if compare_option == 'coins':
        for metric in metrics:
            for coin in coins:
                coin_data = selected_data[selected_data['Crypto'] == coin]
                plt.plot(coin_data.index, coin_data[metric], label=f"{coin} {metric}")
    elif compare_option == 'metrics':
        for coin in coins:
            for metric in metrics:
                coin_data = selected_data[selected_data['Crypto'] == coin]
                plt.plot(coin_data.index, coin_data[metric], label=f"{coin} {metric}")
    else:
        for coin in coins:
            for metric in metrics:
                coin_data = selected_data[selected_data['Crypto'] == coin]
                plt.plot(coin_data.index, coin_data[metric], label=f"{coin} {metric}")

    # Add labels and title
    plt.title('Cryptocurrency Prices Over Time')
    plt.xlabel('Date')
    plt.ylabel('Price/Volume')
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)

    # Show plot
    st.pyplot(plt)

def plot_candlestick_chart(crypto_data, coin='BTC-GBP', period='D'):
    st.title('Cryptocurrency Price Visualization by Interval')
    st.subheader(f'{coin} {period.upper()} Candlestick Chart')
    # Filter the DataFrame for the selected coin
    coin_data = crypto_data[crypto_data['Crypto'] == coin].copy()

    # Make sure the index is a DatetimeIndex
    coin_data.index = pd.to_datetime(coin_data.index)

    # Resample data based on the selected period
    if period.upper() == 'W':
        resampled_data = coin_data.resample('W').agg({'Open': 'first',
                                                      'High': 'max',
                                                      'Low': 'min',
                                                      'Close': 'last',
                                                      'Volume': 'sum'})
    elif period.upper() == 'M':
        resampled_data = coin_data.resample('M').agg({'Open': 'first',
                                                      'High': 'max',
                                                      'Low': 'min',
                                                      'Close': 'last',
                                                      'Volume': 'sum'})
    else:  # Default to daily data if period is not weekly or monthly
        resampled_data = coin_data

    # Plotting
    mpf.plot(resampled_data, type='candle', style='charles', volume=True,
             title=f'{coin} {period.upper()} Candlestick Chart', warn_too_much_data=1000)

def predict_highs_lows(crypto_data):
    """
    Predicts possible highs and lows of a chosen cryptocurrency.

    Parameters:
    - crypto_data (pd.DataFrame): DataFrame containing the cryptocurrency data.

    Returns:
    - predicted_highs (pd.DataFrame): DataFrame indicating possible highs.
    - predicted_lows (pd.DataFrame): DataFrame indicating possible lows.
    """

    # Extract available coins from the data
    available_coins = crypto_data['Crypto'].unique()

    # User input for selecting the coin
    coin_selected = st.sidebar.selectbox("Select the cryptocurrency to analyze", available_coins)

    # Filter data for the selected coin
    selected_data = crypto_data[crypto_data['Crypto'] == coin_selected].copy()

    # Calculate percentiles for highs and lows
    high_threshold = np.percentile(selected_data['High'], 90)
    low_threshold = np.percentile(selected_data['Low'], 10)

    # Predict highs and lows based on thresholds
    predicted_highs = selected_data['Close'] > high_threshold
    predicted_lows = selected_data['Close'] < low_threshold

    # Visualize predicted highs and lows
    st.subheader(f'Predicted Highs and Lows for {coin_selected}')
    plt.figure(figsize=(10, 6))
    plt.plot(selected_data.index, selected_data['Close'], label='Close Price')
    plt.scatter(selected_data.index[predicted_highs], selected_data['Close'][predicted_highs], color='red', label='Predicted Highs')
    plt.scatter(selected_data.index[predicted_lows], selected_data['Close'][predicted_lows], color='green', label='Predicted Lows')
    plt.title(f'Predicted Highs and Lows for {coin_selected}')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    st.pyplot(plt)

    # Format output as DataFrame
    predicted_highs_df = pd.DataFrame({'Date': selected_data.index, 'Predicted Highs': predicted_highs})
    predicted_lows_df = pd.DataFrame({'Date': selected_data.index, 'Predicted Lows': predicted_lows})

    return predicted_highs_df, predicted_lows_df

# Load crypto data (replace this with your own data loading method)
crypto_data = pd.read_csv('crypto_data.csv')

# Sidebar navigation
side_bars = st.sidebar.radio("Navigation", ["Market State", "Price Metrics", "Candlestick Chart", "Predicted Highs and Lows"])

# Conditionals for sidebar navigation
if side_bars == "Market State":
    predict_market_state(crypto_data)
elif side_bars == "Price Metrics":
    # User input
    compare_option = st.sidebar.radio("Select comparison option:", ['coins', 'metrics', 'none'])
    coins = st.sidebar.multiselect("Select cryptocurrencies:", crypto_data['Crypto'].unique(), default=['BTC-GBP'])
    metrics = st.sidebar.multiselect("Select price metrics:", ['Close', 'Open', 'High', 'Low', 'Volume'], default=['Close'])
    start_date = st.sidebar.date_input("Select start date:", pd.to_datetime('2023-04-08'))
    end_date = st.sidebar.date_input("Select end date:", pd.to_datetime('2024-04-08'))

    # Plot the selected cryptocurrency prices
    if st.sidebar.button("Plot Cryptocurrency Prices"):
        plot_crypto_prices_visualization(crypto_data, compare_option, coins, metrics, start_date, end_date)
elif side_bars == "Candlestick Chart":
    # User input for visualization
    coin_visualization = st.sidebar.selectbox("Select a coin for visualization:", crypto_data['Crypto'].unique(), index=0)
    period_visualization = st.sidebar.selectbox("Select period for visualization:", ['D', 'W', 'M'], index=0)

    # Plot the selected cryptocurrency chart
    if st.sidebar.button("Plot Cryptocurrency Chart"):
        plot_candlestick_chart(crypto_data, coin_visualization, period_visualization)
elif side_bars == "Predicted Highs and Lows":
    predict_highs_lows(crypto_data)
